In [4]:
from tqdm import tqdm
from bs4 import BeautifulSoup
import requests
import pickle
import pandas as pd

from joblib import Parallel, delayed

import re
payload = {'si_username': 'ElPresidente',
               'si_password': '6843102t'}
files = []
url_auth="https://www.criticker.com/authenticate.php"
url_edit = "https://www.criticker.com/ratings/?"

s = requests.Session()
response = s.request("POST", url_auth, data=payload, files=files, allow_redirects=False)
def process(row):
    try:
        html_text = s.get(row.url).text
    except:
        return None, None
    soup = BeautifulSoup(html_text, 'html.parser')
    stuff = soup.find("div", class_='crit_container')
    if stuff is None:
        return None, None
    crit_id = stuff.attrs['data-titleid']
    imdb_id = re.findall("title/tt.*/", html_text)[0].split('/')[1]

    return imdb_id, crit_id

In [5]:
df = pd.read_csv('movies.csv')
df = df[df.n_ratings > 115] # remove to process more movies
results = Parallel(n_jobs=8)(delayed(process)(row) for idx, row in tqdm(df.iterrows(), total=len(df)))

100%|█████████████████████████████████████| 12965/12965 [27:46<00:00,  7.78it/s]


In [9]:
films = {}
for film in results:
    if film[0] is not None:
        films[film[0]] = film[1]

In [ ]:
new_films = {"tt4154756": 144842,
"tt6285944": 169133,
"tt15392100": 201102,
"tt7657566": 178429,
"tt1160419": 179798,
"tt3203290": 93202,
"tt8847712": 167704,
"tt4244994": 196554,
"tt0120737": 1840,
"tt0167260": 1841,
"tt0167261": 1842,
"tt10872600": 202657,
"tt6723592": 172576,
"tt11291274": 207140,
"tt11657662": 198369,
"tt6856242": 173863,
"tt12361974": 177935,
"tt9484998": 167373,
"tt3480822": 185001,
"tt10919420": 199929,
"tt9140554": 192106,
"tt4761112": 183237,
"tt10539608": 176333,
"tt1877830": 201448,
"tt6264654": 196065,
"tt2382320": 168984,
"tt1879016": 215264,
"tt9376612": 198026,
"tt6334354": 184382,
"tt6149154": 162021,
"tt7126948": 170577,
"tt4154796": 158692,
"tt7846844": 174500}
films =  {**films, **new_films}

In [15]:
with open('all_films.pkl', 'wb') as f:
    pickle.dump(films, f)